### Een parser-generator voor de wordgrammar

De parser-generator voor de ETCBC-wordgrammar-bestanden is gegenereerd met Yapps2 (zie de website: http://theory.stanford.edu/~amitp/yapps/ en https://github.com/smurfix/yapps) met het grammar-bestand `wgr.g`, wat resulteert in de grammar-parser `wgr.py`. Dat script is afhankelijk van de runtime-module van Yapps, hier meegeleverd als het bestand `yapps-runtime.py`.

Wat we eigenlijk willen is niet het parsen van de `wordgrammar`-bestanden, wat `wgr.py` doet, maar het parsen van de morphologische analyse van de ETCBC-database. Dat gebeurt in `wrdgrm.py`. Dat is, behalve van `wgr.py` en `yapps-runtime.py`, nog afhankelijk van de modules `alphabet.py` en `lexicon.py` (**TODO**: dat moet makkelijker kunnen). Tenslotte zijn de databestanden vereist: het alfabet, het lexicon en de wordgrammar zelf.

In [1]:
# eerst modules importeren
import os, wrdgrm

In [2]:
filepath = lambda rel_path: os.path.realpath(os.path.join(os.getcwd(), rel_path))

In [3]:
# bestandslocaties
alphabet_file = filepath('../../data/wordgrammar/alphabet')
lexicon_file = filepath('../../data/blc/syrlex')
word_grammar_file = filepath('../../data/blc/syrwgr')
an_file = filepath('../../data/blc/Laws.an')

In [4]:
# dan kan de parser worden geïnitialiseerd
w = wrdgrm.wrdgrm(word_grammar_file, lexicon_file, alphabet_file)

De functie `analyze_word()` geeft een analyse als een tuple met drie waarden: `morphemes`, `functions` en `lex`.

De eerste, `morphemes`, bevat een tuple met alle gevonden morfemen, elk als een tuple met drie strings: de paradigmatische vorm (zoals die in het lexicon staat), de oppervlaktevorm (zoals die in de tekst staat), en de geannoteerde vorm met morfologische codering.

De tweede waarde, `functions`, bevat de grammaticale functies van het woord, zoals die in de `wordgrammar` gedefinieerd zijn: `ps: "person"`, `nu: "number"`, `gn: "gender"`, `ls: "lexical set"`, `sp: "part of speech"`, `st: "state"`, `vo: "voice"`, `vs: "verbal stem"`, `vt: "verbal tense"`. Een veld met de waarde `False` geeft aan dat deze functie niet van toepassing is op dit woord, een veld met waarde `None` geeft aan dat de waarde niet is vastgesteld.

De derde waarde, `lex`, bevat het lemma zoals dat in het lexicon staat, met als eerste het woord-id, en vervolgens de annotaties. Behalve standaard-waarden voor de grammaticale functies bevat het lexicon een `gl`-veld voor ieder woord (gloss), en soms een `de`-veld (derived form). (In één resp. twee gevallen komen ook de velden `cs` en `ln` voor, waarvan de betekenis mij niet duidelijk is.)

In [5]:
# voorbeeld
w.analyze_word('>TR/&WT=~>')

((('lex', ('>TR', '>TR', '>TR')),
  ('nme', ('T=', 'WT', '&WT=')),
  ('emf', ('>', '>', '>'))),
 (('vt', False),
  ('vs', False),
  ('ps', False),
  ('sp', 'subs'),
  ('nu', 'pl'),
  ('gn', 'm'),
  ('st', 'emph')),
 ('17789', (('sp', 'subs'), ('gn', 'm'), ('gl', 'place, region'))))

In [6]:
# toon alle beschikbare data uit de an-file:
with open(an_file) as f:
    for line in f:
        verse, s, a = line.split()
        print(verse, s, a)
        for e in a.split('-'):
            morphemes, functions, lex = w.analyze_word(e)
            print(f'\t{morphemes}\n\t{functions}\n\t{lex}')


0,1 TWB TWB
	(('lex', ('TWB', 'TWB', 'TWB')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'advb'))
	('11299', (('sp', 'advb'), ('gl', 'again, back')))
0,1 KTB> KTB=/~>
	(('lex', ('KTB=', 'KTB', 'KTB=')), ('nme', ('', '', '')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', 'm'), ('st', 'emph'))
	('8929', (('sp', 'subs'), ('gn', 'm'), ('gl', 'writing, book')))
0,1 DNM"WS> D-NMWS/(J~>
	(('lex', ('D', 'D', 'D')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'), ('ls', 'pcon'))
	('7789', (('sp', 'prep'), ('ls', 'pcon'), ('gl', '(relative)')))
	(('lex', ('NMWS', 'NMWS', 'NMWS')), ('nme', ('J', '', '(J')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', 'pl'), ('gn', 'm'), ('st', 'emph'))
	('2063', (('sp', 'subs'), ('gn', 'm'), ('gl', 'nome, prefecture, law, custom, us

7,1 DCRR> D-CRR/~>
	(('lex', ('D', 'D', 'D')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'), ('ls', 'pcon'))
	('7789', (('sp', 'prep'), ('ls', 'pcon'), ('gl', '(relative)')))
	(('lex', ('CRR', 'CRR', 'CRR')), ('nme', ('', '', '')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', None), ('st', 'emph'))
	('3581', (('sp', 'subs'), ('gl', 'truth')))
7,1 L> L>
	(('lex', ('L>', 'L>', 'L>')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'nega'))
	('3989', (('sp', 'nega'), ('gl', 'not')))
7,1 QBLW QBL[W:d
	(('lex', ('QBL', 'QBL', 'QBL')), ('vbe', ('W', 'W', 'W')), ('vpm', ('d', 'd', 'd')))
	(('nu', 'pl'), ('gn', 'm'), ('st', False), ('sp', 'verb'), ('vo', 'act'), ('vs', 'pa'), ('vt', 'pf'), ('ps', 'third'))
	('2503', (('sp', 'verb'), ('gl', 'PE to accuse, to impeach, PA to receive, to accept, AF to be opposite, to fa

	(('lex', ('L>', 'L', 'L(>')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'nega'))
	('3989', (('sp', 'nega'), ('gl', 'not')))
	(('lex', ('>JTJ', 'JT', '(>JT(J')), ('nme', ('', '', '')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', None), ('st', None), ('ls', 'nmex'))
	('653', (('sp', 'subs'), ('ls', 'nmex'), ('gl', 'there is')))
8,20 BH B-H
	(('lex', ('B', 'B', 'B')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('10289', (('sp', 'prep'), ('gl', 'in')))
	(('lex', ('H', 'H', 'H')),)
	(('nu', 'sg'), ('gn', 'm'), ('st', False), ('vt', False), ('vs', False), ('ps', 'third'), ('sp', 'pron'), ('ls', 'pers'))
	('23102', (('sp', 'pron'), ('ls', 'pers'), ('ps', 'third'), ('gn', 'm'), ('nu', 'sg'), ('gl', 'he')))
8,20 >JTJ> >JTJ/~>
	(('lex', ('>JTJ', '>JTJ', '>JTJ')), ('nme', ('', '', '')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs

11,7 NGNWB !N!GN&WB=[
	(('pfm', ('N', 'N', 'N')), ('lex', ('GNB=', 'GNWB', 'GN&WB=')), ('vbe', ('', '', '')))
	(('nu', 'sg'), ('gn', 'm'), ('st', False), ('sp', 'verb'), ('vo', 'act'), ('vs', 'pe'), ('vt', 'ipf'), ('ps', 'third'))
	('7589', (('sp', 'verb'), ('gl', 'to steal, to do secretly, to conceal, to deceive')))
11,7 >W >W
	(('lex', ('>W', '>W', '>W')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('20219', (('sp', 'conj'), ('gl', 'or, either')))
11,7 MN MN
	(('lex', ('MN', 'MN', 'MN')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('18481', (('sp', 'prep'), ('gl', 'from')))
11,7 DL> D-L>
	(('lex', ('D', 'D', 'D')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'), ('ls', 'pcon'))
	('7789', (('sp', 'prep'), ('ls', 'pcon'), ('gl', '(relative)')))
	(('lex', ('L>', 'L>', 'L>')),)
	(('nu', False), (

	('20411', ())
14,4 D< !!(JD<[
	(('pfm', ('', '', '')), ('lex', ('JD<', 'D<', '(JD<')), ('vbe', ('', '', '')))
	(('nu', 'sg'), ('gn', 'm'), ('st', False), ('sp', 'verb'), ('vo', 'act'), ('vs', 'pe'), ('vt', 'imp'), ('ps', 'second'))
	('6569', (('sp', 'verb'), ('gl', 'to know, ESHTAPH to know, to recognize, PA to inform, indicate, APH to make known, ETTAPH to be made known, announced')))
14,4 GJR GJR
	(('lex', ('GJR', 'GJR', 'GJR')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('5689', (('sp', 'conj'), ('gl', 'for, but, indeed, however')))
14,4 BRJ BR/-J
	(('lex', ('BR', 'BR', 'BR')), ('nme', ('', '', '')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', 'm'), ('st', None))
	('8101', (('sp', 'subs'), ('gn', 'm'), ('gl', 'male offspring')))
	(('lex', ('J', 'J', 'J')),)
	(('nu', 'sg'), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', 'first'), ('sp', 'pron'), ('ls', 'per

...

36,1 DBJR"WVWNJ> D-B&JR(J&WVWNJ/(J~>
	(('lex', ('D', 'D', 'D')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'), ('ls', 'pcon'))
	('7789', (('sp', 'prep'), ('ls', 'pcon'), ('gl', '(relative)')))
	(('lex', ('BRJVWNJ', 'BJRWVWNJ', 'B&JR(J&WVWNJ')), ('nme', ('J', '', '(J')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'adjv'), ('nu', 'pl'), ('gn', 'm'), ('st', 'emph'), ('ls', 'gntl'))
	('8070', (('sp', 'adjv'), ('ls', 'gntl'), ('gl', 'Brittannian')))
36,2 BJT BJ(N(J&T
	(('lex', ('BJNJ', 'BJT', 'BJ(N(J&T')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('12671', (('sp', 'prep'), ('gl', 'between, among, within')))
36,2 BR"JVWNJ> BRJVWNJ/(J~>
	(('lex', ('BRJVWNJ', 'BRJVWNJ', 'BRJVWNJ')), ('nme', ('J', '', '(J')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'adjv'), ('nu', 'pl'), ('gn', 'm'), ('st'

39,11 WLKWN W-L-KWN==/
	(('lex', ('W', 'W', 'W')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('15227', (('sp', 'conj'), ('gl', 'and')))
	(('lex', ('L', 'L', 'L')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('5881', (('sp', 'prep'), ('gl', 'to')))
	(('lex', ('KWN==', 'KWN', 'KWN==')), ('nme', ('', '', '')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', None), ('st', 'abs'), ('ls', 'prop'))
	('23718', (('sp', 'subs'), ('ls', 'prop'), ('st', 'abs'), ('de', 'K>WN'), ('gl', 'Saturn')))
40,1 >L> >L>
	(('lex', ('>L>', '>L>', '>L>')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('5099', (('sp', 'conj'), ('gl', 'if not, unless')))
40,1 BKLHWN B-KL/-HWN=
	(('lex', ('B', 'B', 'B')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), (

	(('lex', ('KL', 'KWL', 'K&WL')), ('nme', ('', '', '')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', None), ('st', None))
	('7907', (('sp', 'subs'), ('gl', 'whole')))
42,6 R"WXJN RWX/JN
	(('lex', ('RWX', 'RWX', 'RWX')), ('nme', ('JN', 'JN', 'JN')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', 'pl'), ('gn', 'f'), ('st', 'abs'))
	('8779', (('sp', 'subs'), ('gn', 'f'), ('gl', 'breath, wind, spirit')))
42,6 WBKL W-B-KL/
	(('lex', ('W', 'W', 'W')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('15227', (('sp', 'conj'), ('gl', 'and')))
	(('lex', ('B', 'B', 'B')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('10289', (('sp', 'prep'), ('gl', 'in')))
	(('lex', ('KL', 'KL', 'KL')), ('nme', ('', '', '')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', None), ('gn', None), ('st', No

	('15227', (('sp', 'conj'), ('gl', 'and')))
	(('pfm', ('M', 'M', 'M')), ('pfx', ('>T', 'T', '(>T')), ('lex', ('KRH', 'KRH', 'KRH')), ('vbe', ('', '', '')), ('nme', ('JN', 'JN', 'JN')))
	(('sp', 'verb'), ('nu', 'pl'), ('gn', 'm'), ('st', 'abs'), ('vo', 'pas'), ('vs', 'pe'), ('ps', False), ('vt', 'ptc'))
	('16007', (('sp', 'verb'), ('gl', 'PE ptc pss sick, ill weak, ETHPE to fall sick, to be weak, to be tender, PA to make ill, ETHPA to become very ill')))
44,3 WMJ^TJN W-M(W&JT[/JN
	(('lex', ('W', 'W', 'W')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('15227', (('sp', 'conj'), ('gl', 'and')))
	(('lex', ('MWT', 'MJT', 'M(W&JT')), ('vbe', ('', '', '')), ('nme', ('JN', 'JN', 'JN')))
	(('sp', 'verb'), ('nu', 'pl'), ('gn', 'm'), ('st', 'abs'), ('vo', 'act'), ('vs', 'pe'), ('ps', False), ('vt', 'ptc'))
	('2293', (('sp', 'verb'), ('gl', 'to die, AF to put to death')))
44,3 HLJN HLJN
	(('lex', ('HLJN', 'HLJN', 'HLJN')),)
	(('nu',

	(('lex', ('QWM', 'QJM', 'Q(W&JM')), ('vbe', ('', '', '')), ('nme', ('JN', 'JN', 'JN')))
	(('sp', 'verb'), ('nu', 'pl'), ('gn', 'm'), ('st', 'abs'), ('vo', 'act'), ('vs', 'pe'), ('ps', False), ('vt', 'ptc'))
	('17393', (('sp', 'verb'), ('gl', 'to arise APH raise up, place')))
47,5 >L> >L>
	(('lex', ('>L>', '>L>', '>L>')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'conj'))
	('5099', (('sp', 'conj'), ('gl', 'if not, unless')))
47,5 BBJCWT> B-BJCW/T~>
	(('lex', ('B', 'B', 'B')),)
	(('nu', False), ('gn', False), ('st', False), ('vt', False), ('vs', False), ('ps', False), ('sp', 'prep'))
	('10289', (('sp', 'prep'), ('gl', 'in')))
	(('lex', ('BJCW', 'BJCW', 'BJCW')), ('nme', ('T', 'T', 'T')), ('emf', ('>', '>', '>')))
	(('vt', False), ('vs', False), ('ps', False), ('sp', 'subs'), ('nu', 'sg'), ('gn', 'f'), ('st', 'emph'))
	('21661', (('sp', 'subs'), ('de', 'BJCWT>'), ('gn', 'f'), ('gl', 'evil, harm, wickedness, envy')))
47,5 WBVW<JJ W

In [ ]:
# toon output als in dmp-file:
def dump_anfile(an_file):
    with open(an_file) as f:
        for line in f:
            verse, s, a = line.split()
            heading = f'BLC {verse}'
            for e in a.split('-'):
                morphemes, functions, lex = w.analyze_word(e)
                surface_form = ''.join((m[1][1] for m in morphemes if m[0] != 'vpm'))
                lexeme = dict(morphemes)['lex'][0]
                affixes = [m for m in morphemes if m[0] != 'lex'] # TODO affix may not be the right term?
                affix_str = ('-' if not affixes else
                    ','.join((f'{e[0]}="{e[1][0]}"' if e[0] != 'vpm' else f'{e[0]}={e[1][0]}' for e in affixes)))
                func_str = ','.join(('+'+fn if fv is None else fn+'='+fv for fn, fv in functions if fv != False))

                yield '\t'.join((heading, e, surface_form, lexeme, affix_str, func_str))

for line in dump_anfile(an_file):
    print(line)

BLC 0,1	TWB	TWB	TWB	-	sp=advb
BLC 0,1	KTB=/~>	KTB>	KTB=	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
BLC 0,1	D	D	D	-	sp=prep,ls=pcon
BLC 0,1	NMWS/(J~>	NMWS>	NMWS	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 0,1	D	D	D	-	sp=prep,ls=pcon
BLC 0,1	>TR/&WT=~>	>TRWT>	>TR	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 1,1	MN	MN	MN	-	sp=prep
BLC 1,1	QDM	QDM	QDM	-	sp=prep
BLC 1,1	JWM/T=~>	JWMT>	JWM	nme="T=",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 1,1	<L=[/JN	<LJN	<L=	vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pe,vt=ptc
BLC 1,1	HWJ[N	HWJN	HWJ	vbe="N"	nu=pl,sp=verb,vo=act,vs=pe,vt=pf,ps=first,ls=vbex
BLC 1,1	L	L	L	-	sp=prep
BLC 1,1	!M!S<R=[/	MS<R	S<R=	pfm="M",vbe="",nme=""	sp=verb,+st,vo=act,vs=pe,vt=inf
BLC 1,1	L	L	L	-	sp=prep
BLC 1,1	CMCGRM/	CMCGRM	CMCGRM	nme=""	sp=subs,+nu,+gn,st=abs,ls=prop
BLC 1,1	>X/&W	>XW	>X	nme=""	sp=subs,+nu,+gn,+st
BLC 1,1	N	N	N	-	nu=pl,ps=first,sp=pron,ls=pers
BLC 1,1	W	W	W	-	sp=conj
BLC 1,1	>T(J&>[	>T>	>TJ	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third
BLC

BLC 6,1	HW(J&>[	HW>	HWJ	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third,ls=vbex
BLC 6,1	<WJD>/	<WJD>	<WJD>	nme=""	sp=subs,+nu,+gn,st=abs,ls=prop
BLC 6,1	B	B	B	-	sp=prep
BLC 6,1	LXWD/	LXWD	LXWD	nme=""	sp=subs,+nu,+gn,+st
BLC 6,1	L>	L>	L>	-	sp=nega
BLC 6,1	YB(J&>[/	YB>	YBJ	vbe="",nme=""	sp=verb,+nu,+gn,+st,vo=act,vs=pe,vt=ptc
BLC 6,1	L	L	L	-	sp=prep
BLC 6,1	!M!HJMN[/W:d	MHJMNW	HJMN	pfm="M",vbe="",nme="W",vpm=d	sp=verb,st=abs,vo=act,vs=pa,vt=inf
BLC 6,1	>L>	>L>	>L>	-	sp=conj
BLC 6,1	>P	>P	>P	-	sp=conj
BLC 6,1	SGJ>/(J~>	SGJ>>	SGJ>	nme="J",emf=">"	sp=adjv,nu=pl,gn=m,st=emph
BLC 6,2	MVL	MVL	MVL	-	sp=prep
BLC 6,2	D	D	D	-	sp=prep,ls=pcon
BLC 6,2	HJMNW/T~>	HJMNWT>	HJMNW	nme="T",emf=">"	sp=subs,nu=sg,gn=f,st=emph
BLC 6,2	L(>	L	L>	-	sp=nega
BLC 6,2	(>JT(J/	JT	>JTJ	nme=""	sp=subs,+nu,+gn,+st,ls=nmex
BLC 6,2	B	B	B	-	sp=prep
BLC 6,2	HWN=	HWN	HWN=	-	nu=pl,gn=m,ps=third,sp=pron,ls=pers
BLC 6,2	>P	>P	>P	-	sp=conj
BLC 6,2	L>	L>	L>	-	sp=nega
BLC 6,2	L	L	L	-	sp=prep
BLC 6,2	!M!@(>T@&VP(JS[/W	MTVPSW	

BLC 8,5	HJ	HJ	HJ	-	nu=sg,gn=f,ps=third,sp=pron,ls=pers
BLC 8,5	MRKB(>/T~>	MRKBT>	MRKB>	nme="T",emf=">"	sp=subs,nu=sg,gn=f,st=emph
BLC 8,5	>N	>N	>N	-	sp=conj,ls=pint
BLC 8,5	CPJR/	CPJR	CPJR	nme=""	sp=adjv,+nu,+gn,+st
BLC 8,5	!M!@(>T@NGD[/>	MTNGD>	NGD	pfm="M",pfx=">T",vbe="",nme=">"	sp=verb,nu=sg,gn=f,st=abs,vo=pas,vs=pe,vt=ptc
BLC 8,5	W	W	W	-	sp=conj
BLC 8,5	!M!@(>T@DBR[/>:d	MTDBR>	DBR	pfm="M",pfx=">T",vbe="",nme=">",vpm=d	sp=verb,nu=sg,gn=f,st=abs,vo=pas,vs=pa,vt=ptc
BLC 8,5	>L>	>L>	>L>	-	sp=conj
BLC 8,5	M>N/(J~>	M>N>	M>N	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 8,5	>NWN	>NWN	>NWN	-	nu=pl,gn=m,ps=third,sp=pron,ls=pers
BLC 8,5	D	D	D	-	sp=prep,ls=pcon
BLC 8,5	<B&JD[/JN:p	<BJDJN	<BD	vbe="",nme="JN",vpm=p	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pe,vt=ptc
BLC 8,5	L	L	L	-	sp=prep
BLC 8,5	XWCX/~>	XWCX>	XWCX	nme="",emf=">"	sp=subs,+nu,+gn,st=emph
BLC 8,5	D	D	D	-	sp=prep,ls=pcon
BLC 8,5	HW=	HW	HW=	-	nu=sg,gn=m,sp=pron,ls=demo
BLC 8,5	MN=	MN	MN=	-	sp=pron,ls=prin
BLC 8,5	D	D	D	-	sp=prep,ls=pco

BLC 10,1	H	H	H	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 10,1	W	W	W	-	sp=conj
BLC 10,1	>JKN&>	>JKN>	>JKN	-	sp=advb,ls=pint
BLC 10,1	>JLJN	>JLJN	>JLJN	-	nu=pl,sp=pron,ls=prin
BLC 10,1	D	D	D	-	sp=prep,ls=pcon
BLC 10,1	QB&J<[/N:p	QBJ<N	QB<	vbe="",nme="N",vpm=p	sp=verb,nu=pl,gn=f,st=abs,vo=pas,vs=pe,vt=ptc
BLC 10,1	>NJN	>NJN	>NJN	-	nu=pl,gn=f,ps=third,sp=pron,ls=pers
BLC 10,1	!M!@(>T@D(W&JN[/N	MTDJNN	DWN	pfm="M",pfx=">T",vbe="",nme="N"	sp=verb,nu=pl,gn=f,st=abs,vo=pas,vs=pe,vt=ptc
BLC 10,2	>MR[	>MR	>MR	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third,ls=quot
BLC 10,2	L	L	L	-	sp=prep
BLC 10,2	J	J	J	-	nu=sg,ps=first,sp=pron,ls=pers
BLC 10,2	L>	L>	L>	-	sp=nega
BLC 10,2	HW(J&>[	HW>	HWJ	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third,ls=vbex
BLC 10,2	B	B	B	-	sp=prep
BLC 10,2	HD>	HD>	HD>	-	nu=sg,gn=f,sp=pron,ls=demo
BLC 10,2	D	D	D	-	sp=prep,ls=pcon
BLC 10,2	QB&J<[/JN:p	QBJ<JN	QB<	vbe="",nme="JN",vpm=p	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pe,vt=ptc
BLC 10,2	>W=	>W	>W=	-	sp=intj
BLC 10,2

BLC 11,6	R<(J=[/JN	R<JN	R<J=	vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pe,vt=ptc
BLC 11,6	(X(NN	N	XNN	-	nu=pl,ps=first,sp=pron,ls=pers
BLC 11,6	D	D	D	-	sp=prep,ls=pcon
BLC 11,6	>P	>P	>P	-	sp=conj
BLC 11,6	L	L	L	-	sp=prep
BLC 11,6	N	N	N	-	nu=pl,ps=first,sp=pron,ls=pers
BLC 11,6	HKWT	HKWT	HKWT	-	sp=advb
BLC 11,6	!N!HW(J&>[	NHW>	HWJ	pfm="N",vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=ipf,ps=third,ls=vbex
BLC 11,7	MN=	MN	MN=	-	sp=pron,ls=prin
BLC 11,7	(HW	W	HW	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 11,7	HKJL	HKJL	HKJL	-	sp=conj
BLC 11,7	BR/	BR	BR	nme=""	sp=subs,+nu,gn=m,+st
BLC 11,7	(>NC/~>	NC>	>NC	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
BLC 11,7	D	D	D	-	sp=prep,ls=pcon
BLC 11,7	MXJL/	MXJL	MXJL	nme=""	sp=adjv,+nu,+gn,+st
BLC 11,7	MN	MN	MN	-	sp=prep
BLC 11,7	D	D	D	-	sp=prep,ls=pcon
BLC 11,7	L>	L>	L>	-	sp=nega
BLC 11,7	!N!GN&WB=[	NGNWB	GNB=	pfm="N",vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=ipf,ps=third
BLC 11,7	>W	>W	>W	-	sp=conj
BLC 11,7	MN	MN	MN	-	sp=prep
BLC 11,7	D	D	D	-	sp=pre

...

BLC 21,6	!M!@(>T@XBL[/JN	MTXBLJN	XBL	pfm="M",pfx=">T",vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pe,vt=ptc
BLC 21,6	VLJ/(J~>	VLJ>	VLJ	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 21,6	W	W	W	-	sp=conj
BLC 21,6	B	B	B	-	sp=prep
BLC 21,6	ZBN/	ZBN	ZBN	nme=""	sp=subs,+nu,gn=m,+st
BLC 21,6	!M!@(>(T&C@(C&TD(J[/JN	MCTDJN	CDJ	pfm="M",pfx=">T",vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pe,vt=ptc
BLC 21,6	W	W	W	-	sp=conj
BLC 21,6	B	B	B	-	sp=prep
BLC 21,6	ZBN/	ZBN	ZBN	nme=""	sp=subs,+nu,gn=m,+st
BLC 21,6	M(W&JT[/JN	MJTJN	MWT	vbe="",nme="JN"	sp=verb,nu=pl,gn=m,st=abs,vo=act,vs=pe,vt=ptc
BLC 21,6	D	D	D	-	sp=prep,ls=pcon
BLC 21,6	L>	L>	L>	-	sp=nega
BLC 21,6	B	B	B	-	sp=prep
BLC 21,6	ZBN/	ZBN	ZBN	nme=""	sp=subs,+nu,gn=m,+st
BLC 21,6	HWN=	HWN	HWN=	-	nu=pl,gn=m,ps=third,sp=pron,ls=pers
BLC 21,7	MN	MN	MN	-	sp=prep
BLC 21,7	KJN/~>	KJN>	KJN	nme="",emf=">"	sp=subs,+nu,+gn,st=emph
BLC 21,7	HWJ[/>	HWJ>	HWJ	vbe="",nme=">"	sp=verb,nu=sg,gn=f,st=abs,vo=act,vs=pe,vt=ptc,ls=vbex
BLC 21,7	SB<W/T~>

BLC 25,8	>N>	>N>	>N>	-	nu=sg,ps=first,sp=pron,ls=pers
BLC 25,8	MN	MN	MN	-	sp=prep
BLC 25,8	MDNX/~>	MDNX>	MDNX	nme="",emf=">"	sp=subs,+nu,+gn,st=emph
BLC 25,8	R(JC/	RC	RJC	nme=""	sp=subs,+nu,gn=m,+st
BLC 25,8	H	H	H	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 25,8	D	D	D	-	sp=prep,ls=pcon
BLC 25,8	<LM/~>	<LM>	<LM	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
BLC 25,8	KL/	KL	KL	nme=""	sp=subs,+nu,+gn,+st
BLC 25,8	H	H	H	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 26,1	NMWS/(J~>	NMWS>	NMWS	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 26,1	D	D	D	-	sp=prep,ls=pcon
BLC 26,1	C(JRJ/(J~>	CRJ>	CJRJ	nme="J",emf=">"	sp=adjv,nu=pl,gn=m,st=emph,ls=gntl
BLC 26,2	>JT(J/	>JT	>JTJ	nme=""	sp=subs,+nu,+gn,+st,ls=nmex
BLC 26,2	NMWS/(J~>	NMWS>	NMWS	nme="J",emf=">"	sp=subs,nu=pl,gn=m,st=emph
BLC 26,2	L	L	L	-	sp=prep
BLC 26,2	CJRJ/(J~>	CJRJ>	CJRJ	nme="J",emf=">"	sp=adjv,nu=pl,gn=m,st=emph,ls=gntl
BLC 26,2	D	D	D	-	sp=prep,ls=pcon
BLC 26,2	L>	L>	L>	-	sp=nega
BLC 26,2	!N!QVL[WN	NQVLWN	QVL	pfm="N",vbe="WN"	nu=pl,gn=m,sp=ver

BLC 31,5	HJN	HJN	HJN	-	nu=pl,gn=f,ps=third,sp=pron,ls=pers
BLC 31,5	B<L/J	B<LJ	B<L	nme="J"	sp=subs,nu=pl,gn=m,st=cst
BLC 31,5	HJN	HJN	HJN	-	nu=pl,gn=f,ps=third,sp=pron,ls=pers
BLC 31,5	W	W	W	-	sp=conj
BLC 31,5	L>	L>	L>	-	sp=nega
BLC 31,5	DXL(>/T~>	DXLT>	DXL>	nme="T",emf=">"	sp=subs,nu=sg,gn=f,st=emph
BLC 31,5	>JT(J/	>JT	>JTJ	nme=""	sp=subs,+nu,+gn,+st,ls=nmex
BLC 31,5	LWT	LWT	LWT	-	sp=prep
BLC 31,5	HJN	HJN	HJN	-	nu=pl,gn=f,ps=third,sp=pron,ls=pers
BLC 31,5	MVL	MVL	MVL	-	sp=prep
BLC 31,5	D	D	D	-	sp=prep,ls=pcon
BLC 31,5	>JK	>JK	>JK	-	sp=prep,ls=pcon
BLC 31,5	D	D	D	-	sp=prep,ls=pcon
BLC 31,5	L	L	L	-	sp=prep
BLC 31,5	MR(>=/T=~>	MRT>	MR>=	nme="T=",emf=">"	sp=subs,nu=pl,gn=f,st=emph
BLC 31,5	>X&JD[/JN:p	>XJDJN	>XD	vbe="",nme="JN",vpm=p	sp=verb,nu=pl,gn=m,st=abs,vo=pas,vs=pe,vt=ptc
BLC 31,5	L	L	L	-	sp=prep
BLC 31,5	HJN	HJN	HJN	-	nu=pl,gn=f,ps=third,sp=pron,ls=pers
BLC 31,5	QCNJ/(J~>	QCNJ>	QCNJ	nme="J",emf=">"	sp=adjv,nu=pl,gn=m,st=emph,ls=gntl
BLC 31,5	L	L	L	-	sp=prep
BLC 31,5	NC=/J	NCJ	NC=	

BLC 39,11	YBW/T~>	YBWT>	YBW	nme="T",emf=">"	sp=subs,nu=sg,gn=f,st=emph
BLC 39,11	D	D	D	-	sp=prep,ls=pcon
BLC 39,11	KL/	KL	KL	nme=""	sp=subs,+nu,+gn,+st
BLC 39,11	HWN=	HWN	HWN=	-	nu=pl,gn=m,ps=third,sp=pron,ls=pers
BLC 39,11	GRMNJ/(J~>	GRMNJ>	GRMNJ	nme="J",emf=">"	sp=adjv,nu=pl,gn=m,st=emph,ls=gntl
BLC 39,11	!N!HW(J&>[	NHW>	HWJ	pfm="N",vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=ipf,ps=third,ls=vbex
BLC 39,11	S(W&JM[/:p	SJM	SWM	vbe="",nme="",vpm=p	sp=verb,+nu,+gn,+st,vo=pas,vs=pe,vt=ptc
BLC 39,11	BJT/	BJT	BJT	nme=""	sp=subs,+nu,gn=m,+st
BLC 39,11	JLD=/J	JLDJ	JLD=	nme="J"	sp=subs,nu=pl,gn=m,st=cst
BLC 39,11	HWN=	HWN	HWN=	-	nu=pl,gn=m,ps=third,sp=pron,ls=pers
BLC 39,11	SHR/~>	SHR>	SHR	nme="",emf=">"	sp=subs,+nu,+gn,st=emph
BLC 39,11	W	W	W	-	sp=conj
BLC 39,11	C</T~>	C<T>	C<	nme="T",emf=">"	sp=subs,nu=sg,gn=f,st=emph
BLC 39,11	BJNJ	BJNJ	BJNJ	-	sp=prep
BLC 39,11	>RS/	>RS	>RS	nme=""	sp=subs,+nu,+gn,st=abs,ls=prop
BLC 39,11	W	W	W	-	sp=conj
BLC 39,11	L	L	L	-	sp=prep
BLC 39,11	KWN==/	KWN	KWN==	nme

BLC 45,1	W	W	W	-	sp=conj
BLC 45,1	KD	KD	KD	-	sp=conj
BLC 45,1	HJMN[	HJMN	HJMN	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third
BLC 45,1	>BGR/	>BGR	>BGR	nme=""	sp=subs,+nu,gn=m,st=abs,ls=prop
BLC 45,1	MLK/~>	MLK>	MLK	nme="",emf=">"	sp=subs,+nu,gn=m,st=emph
BLC 45,1	PQD[	PQD	PQD	vbe=""	nu=sg,gn=m,sp=verb,vo=act,vs=pe,vt=pf,ps=third
BLC 45,1	D	D	D	-	sp=prep,ls=pcon
BLC 45,1	K&WL/	KWL	KL	nme=""	sp=subs,+nu,+gn,+st
BLC 45,1	D	D	D	-	sp=prep,ls=pcon
BLC 45,1	PSQ[/	PSQ	PSQ	vbe="",nme=""	sp=verb,+nu,+gn,+st,vo=act,vs=pe,vt=ptc
BLC 45,1	GBRW/T	GBRWT	GBRW	nme="T"	sp=subs,nu=sg,gn=f,st=cst
BLC 45,1	H	H	H	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 45,1	!N!PSQ[WN	NPSQWN	PSQ	pfm="N",vbe="WN"	nu=pl,gn=m,sp=verb,vo=act,vs=pe,vt=ipf,ps=third
BLC 45,1	&>JD/	>JD	JD	nme=""	sp=subs,+nu,gn=f,+st
BLC 45,1	H	H	H	-	nu=sg,gn=m,ps=third,sp=pron,ls=pers
BLC 45,2	W	W	W	-	sp=conj
BLC 45,2	MN	MN	MN	-	sp=prep
BLC 45,2	HW=	HW	HW=	-	nu=sg,gn=m,sp=pron,ls=demo
BLC 45,2	JWM/~>	JWM>	JWM	nme="",emf=">"	sp=subs,+nu,gn=m,

Om te controleren dat de output correct is heb ik bovenstaande output vergeleken met de bestaande .dmp-bestanden. Omdat de volgorde van de waarden willekeurig lijkt te zijn - of in ieder geval niet in alle gevallen gelijk - moeten alle waarden gesorteerd worden voor ze vergeleken kunnen worden, een eenvoudige diff volstaat niet. Onderstaand script bevestigt dat bovenstaande output, op de volgorde na, een exacte weergave is van de bestaande .dmp-bestanden:

In [ ]:
dmp_file = filepath('../../data/blc/Laws.dmp')
dmp_gen = dump_anfile(an_file)

with open(dmp_file) as f_dmp:
    for line1, line2 in zip(f_dmp, dmp_gen):
        for f1, f2 in zip(line1.strip().split('\t'), line2.split('\t')):
            f1s, f2s = (','.join(sorted(f.split(','))) for f in (f1,f2))
            if f1s != f2s:
                print(f'{line1}!=\n{line2}')